# Previsão com Prophet
Prof. Fernando Amaral

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from fbprophet import Prophet
#conda install -c conda-forge fbprophet

In [2]:
import matplotlib.pyplot as plt

In [3]:
N = 60 
i = 1800 

In [4]:
all_data = pd.read_csv("../../Dados/train_1.csv").T 
key = pd.read_csv("../../Dados/key_1.csv")

In [5]:
train, test = all_data.iloc[0:-N,:], all_data.iloc[-N:,:]
test_cleaned = test.T.fillna(method='ffill').T
train_cleaned = train.T.iloc[:,1:].fillna(method='ffill').T

In [6]:
data=train_cleaned.iloc[:,i].to_frame()
data.columns = ['visits']
data['mean'] = pd.Series(data.visits).rolling(window=50).mean()
std_mult = 1.5
data.loc[np.abs(data.visits-data.visits.mean())>=(std_mult*data.visits.std()),'visits'] = data.loc[np.abs(data.visits-data.visits.mean())>=(std_mult*data.visits.std()),'mean']
data.index = pd.to_datetime(data.index)

In [7]:
data.head()

visits  mean
2015-07-01     1.0   NaN
2015-07-02     6.0   NaN
2015-07-03     8.0   NaN
2015-07-04     7.0   NaN
2015-07-05    12.0   NaN

In [8]:
X = pd.DataFrame(index=range(0,len(data)))
X['ds'] = data.index
X['y'] = data['visits'].values
print((X.head()))
print(len(X))

          ds     y
0 2015-07-01   1.0
1 2015-07-02   6.0
2 2015-07-03   8.0
3 2015-07-04   7.0
4 2015-07-05  12.0
490


In [ ]:
m = Prophet(yearly_seasonality=True)
m.fit(X)
future = m.make_future_dataframe(periods=N)
future.tail()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
m.plot(forecast);